In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import json
import regex
import unidecode
from itertools import chain

In [2]:
# Load data
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/insiders.parquet")
df_in = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/outsiders.parquet")
df_ou = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/minors.parquet")
df_mi = pd.read_parquet(dir_df)

In [3]:
print(df_in.shape)
print(df_ou.shape)
print(df_mi.shape)

(519520, 195)
(236210, 43)
(1650869, 107)


In [4]:
# Load info from gencat
tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")

C:\Users\josea\AppData\Local\Temp\ipykernel_30660\694994485.py:2: DtypeWarning: Columns (27,43,52,53,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")


In [5]:
# with open(r"C:\Users\josea\Downloads\genCat_Junio_2023.json", "r") as f:
#     gencat = pd.json_normalize(json.load(f))

# Aux functions

In [8]:
from src.utils.utils import evaluate_cell, fill_na, unify_colname


def process_str(el: str):
    s = regex.sub(r"(^[^\w({[]*)|([^\w)}\]]*$)|([^\w:)(/-\s])", "", el).lower().strip()
    s = regex.sub(r"\s+", " ", s)
    return s


def process_cpv(el: str):
    return regex.sub(r"\D", "", el)

In [7]:
# Get general tender info (title, object, winner, etc)
df_in.columns = [unify_colname(c) for c in df_in.columns]
df_ou.columns = [unify_colname(c) for c in df_ou.columns]
df_mi.columns = [unify_colname(c) for c in df_mi.columns]

In [12]:
use_cols = [
    "id",
    "summary",
    "title",
    "updated",
    # "deleted_on",
    "ContractFolderStatus.ContractFolderID",
    "ContractFolderStatus.ContractFolderStatusCode",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyIdentification.ID",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyName.Name",
    "ContractFolderStatus.ProcurementProject.Name",
    "ContractFolderStatus.ProcurementProject.TypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TaxExclusiveAmount",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.DurationMeasure",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.StartDate",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.EndDate",
    # "ContractFolderStatus.TenderResult.ResultCode",
    # "ContractFolderStatus.TenderResult.ReceivedTenderQuantity",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
    "ContractFolderStatus.TenderResult.WinningParty.PartyName.Name",
    # "ContractFolderStatus.TenderResult.AwardedTenderedProject.LegalMonetaryTotal.TaxExclusiveAmount",
    # "ContractFolderStatus.TenderingProcess.ProcedureCode",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndDate",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndTime",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.Description",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod",
    # "ContractFolderStatus.ProcurementProject.SubTypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TotalAmount",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.IdentificationCode",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.Name",
    # "ContractFolderStatus.TenderResult.Description",
    # "ContractFolderStatus.TenderResult.AwardDate",
    # "ContractFolderStatus.TenderResult.StartDate",
    # "ContractFolderStatus.TenderResult.Contract.ID",
    # "ContractFolderStatus.TenderResult.Contract.IssueDate",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.CityName",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.PostalZone",
    "ContractFolderStatus.TenderingTerms.FundingProgramCode",
    "ContractFolderStatus.TenderingTerms.FundingProgram",
    "ContractFolderStatus.TenderResult.WinningParty.PartyLegalEntity.CompanyTypeCode",
]

In [13]:
rename_columns = {
    "title": "title",
    "summary": "summary",
    "ContractFolderStatus.ContractFolderID": "id",
    "ContractFolderStatus.ProcurementProject.Name": "project",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID": "winner",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity": "location",
    "denominacio": "title",
    "objecte_contracte": "summary",
    "codi_expedient": "id",
    "codi_cpv": "cpv",
}

# PLACE

In [14]:
df = pd.concat(
    [
        df_in[[c for c in use_cols if c in df_in.columns]],
        df_ou[[c for c in use_cols if c in df_ou.columns]],
        # df_mi[[c for c in use_cols if c in df_mi.columns]],
    ]
)
index_names = df.index.names
df.reset_index(inplace=True)
df["identifier"] = df[index_names].astype(str).agg("/".join, axis=1)
# df.drop(index_names, inplace=True, axis=1)
df.set_index("identifier", inplace=True)
df = df.applymap(fill_na, fill=np.nan)
df.shape

(755730, 24)

### Get relevant data

In [87]:
# Get only relevant data
tender_filt = (
    df[
        [
            "title",
            "summary",
            "ContractFolderStatus.ContractFolderID",
            # "ContractFolderStatus.ProcurementProject.Name",
            # "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
            "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
            "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
        ]
    ]
    .rename(columns=rename_columns)
    .dropna(how="all")
    .explode("cpv")
    # .explode("winner")
)
tender_filt["id_orig"] = tender_filt["id"]
tender_filt["id_norm"] = tender_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
# tender_filt["winner"] = tender_filt["winner"].apply(evaluate)
tender_filt["cpv"] = tender_filt["cpv"].apply(evaluate_cell)
tender_filt = tender_filt.explode("cpv")  # .explode("winner")
tender_filt["cpv"] = tender_filt["cpv"].astype(str).apply(process_cpv)
tender_filt["location"] = tender_filt["location"].apply(
    lambda x: unidecode.unidecode(x.lower()) if not pd.isna(x) else np.nan
)
tender_filt[[c for c in tender_filt.columns if c != "id"]] = (
    tender_filt[[c for c in tender_filt.columns if c != "id"]]
    .astype(str)
    .applymap(process_str)
)
tender_filt = tender_filt.replace({"": np.nan, "0": np.nan})
tender_filt["cpv_div"] = tender_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)
tender_filt = tender_filt.reset_index()

### Get data from cat

In [122]:
# Filter using only those from Cat
agg_tender_orig = (
    tender_filt[
        tender_filt["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .drop("identifier", axis=1)
    .groupby(["id_orig"])
    .agg(list)
)
valid_agg_tender_orig = agg_tender_orig[
    agg_tender_orig["title"].apply(lambda x: len(set(x)) == 1)
]
valid_agg_tender_orig = valid_agg_tender_orig.applymap(
    lambda x: Counter(x).most_common()[0][0]
)
valid_agg_tender_orig = valid_agg_tender_orig.reset_index().reset_index()
valid_agg_tender_orig["index"] = valid_agg_tender_orig["index"].apply(lambda x: [x])
print(valid_agg_tender_orig.shape)
display(valid_agg_tender_orig.head())

(252111, 9)


,index,id_orig,title,summary,id,cpv,location,id_norm,cpv_div
0,[0],(/2021,ampliació de lescola de freginals per a dispos...,id licitación: (/2021 órgano de contratación: ...,(/2021,452141000,nan,2021,45
1,[1],(900243/22),contracte de serveis per a laixecament topogrà...,id licitación: (900243/22) órgano de contratac...,(900243/22),713518100,nan,900243-22,71
2,[2],(902472/21),contracte de serveis per a laixecament topogrà...,id licitación: (902472/21) órgano de contratac...,(902472/21),713518100,nan,902472-21,71
3,[3],(902531/21),contracte de serveis per a laixecament topogrà...,id licitación: (902531/21) órgano de contratac...,(902531/21),713518100,nan,902531-21,71
4,[4],(902766/21),contracte de serveis per a laixecament topogrà...,id licitación: (902766/21) órgano de contratac...,(902766/21),713518100,nan,902766-21,71


# GENCAT

In [82]:
# Clean tenders cat
tend_cat_filt = (
    tend_cat[
        [
            "denominacio",
            "objecte_contracte",
            "codi_expedient",
            "codi_cpv",
        ]
    ]
    .rename(columns=rename_columns)
    .astype(str)
)
tend_cat_filt["id_orig"] = tend_cat_filt["id"]
tend_cat_filt["id_norm"] = tend_cat_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
tend_cat_filt["cpv"] = tend_cat_filt["cpv"].astype(str).apply(process_cpv)
tend_cat_filt[[c for c in tend_cat_filt if c != "id"]] = (
    tend_cat_filt[[c for c in tend_cat_filt if c != "id"]]
    .astype(str)
    .applymap(process_str)
)
tend_cat_filt = tend_cat_filt.replace({"": np.nan, "0": np.nan})
tend_cat_filt["cpv_div"] = tend_cat_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)
tend_cat_filt = tend_cat_filt.reset_index(drop=True)

In [119]:
# Aggregate by original ID
valid_tend_cat_orig = tend_cat_filt.groupby("id_orig").agg(list)
valid_tend_cat_orig = valid_tend_cat_orig[
    valid_tend_cat_orig["title"].apply(lambda x: len(set(x)) == 1)
]
valid_tend_cat_orig = valid_tend_cat_orig.applymap(
    lambda x: Counter(x).most_common()[0][0]
)
valid_tend_cat_orig = valid_tend_cat_orig.reset_index().reset_index()
valid_tend_cat_orig["index"] = valid_tend_cat_orig["index"].apply(lambda x: [x])
print(len(valid_tend_cat_orig))
display(valid_tend_cat_orig.head())

232124


,index,id_orig,title,summary,id,cpv,id_norm,cpv_div
0,[0],(/2021,ampliacióde lescola de freginals i millora de ...,ampliació de lescola de freginals per a dispos...,(/2021,452141001,2021,45
1,[1],(2) 49/2021/cmsubm2,compra del vestuari i dels equips de protecció...,compra del vestuari i dels equips de protecció...,(2) 49/2021/CMSUBM2,181100003,2-49-2021-cmsubm2,18
2,[2],(2)120/2021/cmsubm1,subministrament de productes dhorticultura i v...,contracte de submnistrament de productes dhort...,(2)120/2021/CMSUBM1,031200008,2-120-2021-cmsubm1,03
3,[3],(900243/22),contracte de serveis per a laixecament topogrà...,contracte de serveis per a laixecament topogrà...,(900243/22),713518104,900243-22,71
4,[4],(902472/21),contracte de serveis per a laixecament topogrà...,contracte de serveis per a laixecament topogrà...,(902472/21),713518104,902472-21,71


# TODO
- Hacer match con budget

# Match PLACE & GENCAT

In [123]:
# Match tenders using orig id
use_tend_agg_orig = valid_agg_tender_orig[["id_orig", "index", "title", "cpv_div"]]
use_tend_cat_orig = valid_tend_cat_orig[["id_orig", "index", "title", "cpv_div"]]

### By ID

In [124]:
# Match tenders by ID orig
matched_id_orig = pd.merge(
    use_tend_agg_orig,
    use_tend_cat_orig,
    how="inner",
    left_on=["id_orig"],
    right_on=["id_orig"],
    suffixes=["_agg", "_cat"],
).reset_index(drop=True)
matched_id_orig["title"] = matched_id_orig[["title_agg", "title_cat"]].apply(
    lambda x: Counter(list(x)), axis=1
)
matched_id_orig["cpv_div"] = matched_id_orig[["cpv_div_agg", "cpv_div_cat"]].apply(
    lambda x: Counter(list(x)), axis=1
)
matched_id_orig = matched_id_orig[
    ["id_orig", "title", "cpv_div", "index_agg", "index_cat"]
]
idx_agg_orig = set(chain.from_iterable(matched_id_orig["index_agg"]))
idx_cat_orig = set(chain.from_iterable(matched_id_orig["index_cat"]))

# Stats
num_agg_idx_found = len(idx_agg_orig)
num_cat_idx_found = len(idx_cat_orig)
num_agg_idx_original = len(use_tend_agg_orig)
num_cat_idx_original = len(use_tend_cat_orig)

print("Indices from PLACE matched with id:")
print(
    f"{num_agg_idx_found} out of {num_agg_idx_original} ({num_agg_idx_found/num_agg_idx_original*100:.3f}%)"
)
print("Indices from GENCAT matched with id:")
print(
    f"{num_cat_idx_found} out of {num_cat_idx_original} ({num_cat_idx_found/num_cat_idx_original*100:.3f}%)"
)
display(matched_id_orig.head())

Indices from PLACE matched with id:
91798 out of 252111 (36.412%)
Indices from GENCAT matched with id:
91798 out of 232124 (39.547%)


,id_orig,title,cpv_div,index_agg,index_cat
0,(/2021,{'ampliació de lescola de freginals per a disp...,{'45': 2},[0],[0]
1,(900243/22),{'contracte de serveis per a laixecament topog...,{'71': 2},[1],[3]
2,(902472/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[2],[4]
3,(902531/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[3],[5]
4,(902766/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[4],[6]


### By title and CPV

In [126]:
# Match tenders by title and cpv orig
matched_title_orig = pd.merge(
    use_tend_agg_orig,
    use_tend_cat_orig,
    how="inner",
    left_on=[
        "title",
        "cpv_div",
    ],
    right_on=[
        "title",
        "cpv_div",
    ],
    suffixes=["_agg", "_cat"],
).reset_index(drop=True)

matched_title_orig = matched_title_orig[
    matched_title_orig["id_orig_agg"] != matched_title_orig["id_orig_cat"]
]

# Split to get those ids that only appear in agg or cat
m_x = matched_title_orig.loc[
    ~matched_title_orig["id_orig_agg"].isin(matched_title_orig["id_orig_cat"]),
    ["id_orig_agg", "index_agg", "title", "cpv_div"],
].rename({"id_orig_agg": "id_orig"}, axis=1)
m_y = matched_title_orig.loc[
    ~matched_title_orig["id_orig_cat"].isin(matched_title_orig["id_orig_agg"]),
    ["id_orig_cat", "index_cat", "title", "cpv_div"],
].rename({"id_orig_cat": "id_orig"}, axis=1)

# Group by id
matched_title_orig = (
    pd.concat([m_x, m_y])
    .groupby("id_orig")
    .agg(
        {
            "title": Counter,
            "cpv_div": Counter,
            "index_agg": sum,
            "index_cat": sum,
        }
    )
    .reset_index()
    .replace({0: None})
)

# Stats
num_agg_title_found = len(matched_title_orig["index_agg"].explode().dropna())
num_cat_title_found = len(matched_title_orig["index_cat"].explode().dropna())

print("Indices from PLACE matched with title:")
print(
    f"{num_agg_title_found} out of {num_agg_idx_original} ({num_agg_title_found/num_agg_idx_original*100:.3f}%)"
)
print("Indices from GENCAT matched with title:")
print(
    f"{num_cat_title_found} out of {num_cat_idx_original} ({num_cat_title_found/num_cat_idx_original*100:.3f}%)"
)
print(
    "The found indices are only present in PLACE or GENCAT, but they match by title, so we asume they are the same and assign the id."
)
display(matched_title_orig.head())

Indices from PLACE matched with title:
14756 out of 252111 (5.853%)
Indices from GENCAT matched with title:
1569 out of 232124 (0.676%)
The found indices are only present in PLACE or GENCAT, but they match by title, so we asume they are the same and assign the id.


,id_orig,title,cpv_div,index_agg,index_cat
0,001/21001877,{'gestió de la casa de laigua de trinitat nova...,{'98': 1},None,[882]
1,001/21002039,{'servei de suport i assistència tècnica pel c...,{'71': 1},None,[894]
2,001/21003081,{'subministrament de material de conservació i...,{'30': 1},[2354],None
3,001/21004872,{'gestió i dinamització del centre cívic torre...,{'98': 1},None,[985]
4,001/22,{'contractació dun servei que inclogui 3 perso...,{'72': 1},None,[989]


### All (id, name, cpv)

In [127]:
# a = pd.concat(
#     [matched_id_orig, matched_title_orig],
# )
# a = a[a["id_orig"].duplicated(keep=False)].sort_values(by="id_orig")
# a.dropna(subset="index_agg")[a["index_agg"].dropna().apply(len) > 1]

In [128]:
# Combine both
matched_final_orig = pd.merge(
    matched_id_orig,
    matched_title_orig,
    how="outer",
    left_on="id_orig",
    right_on="id_orig",
)

title = []
cpv_div = []
index_agg = []
index_cat = []
for tx, ty, cx, cy, iax, iay, icx, icy in matched_final_orig[
    [
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
        "index_agg_x",
        "index_agg_y",
        "index_cat_y",
        "index_cat_x",
    ]
].values:
    t = dict()
    c = dict()
    ia = []
    ic = []
    t.update(tx if not pd.isna(tx) else dict())
    t.update(ty if not pd.isna(ty) else dict())
    c.update(cx if not pd.isna(cx) else dict())
    c.update(cy if not pd.isna(cy) else dict())
    ia.extend(iax if isinstance(iax, list) else [])
    ia.extend(iay if isinstance(iay, list) else [])
    ic.extend(icx if isinstance(icx, list) else [])
    ic.extend(icy if isinstance(icy, list) else [])
    title.append(t)
    cpv_div.append(c)
    index_agg.append(ia)
    index_cat.append(ic)

matched_final_orig["title"] = title
matched_final_orig["cpv_div"] = cpv_div
matched_final_orig["index_agg"] = index_agg
matched_final_orig["index_cat"] = index_cat
matched_final_orig["index_agg"] = matched_final_orig["index_agg"].apply(
    lambda x: list(set(x))
)
matched_final_orig["index_cat"] = matched_final_orig["index_cat"].apply(
    lambda x: list(set(x))
)

matched_final_orig = matched_final_orig[
    ["id_orig", "title", "cpv_div", "index_agg", "index_cat"]
]


# Stats
num_matched = len(matched_final_orig)
num_agg_idx_found = len(matched_final_orig["index_agg"].explode().dropna())
num_cat_idx_found = len(matched_final_orig["index_cat"].explode().dropna())

print(f"Total number of matches: {num_matched}")
print("Indices from PLACE matched with id:")
print(
    f"{num_agg_idx_found} out of {num_agg_idx_original} ({num_agg_idx_found/num_agg_idx_original*100:.3f}%)"
)
print("Indices from GENCAT matched with id:")
print(
    f"{num_cat_idx_found} out of {num_cat_idx_original} ({num_cat_idx_found/num_cat_idx_original*100:.3f}%)"
)

display(matched_final_orig.head())

Total number of matches: 92274
Indices from PLACE matched with id:
92008 out of 252111 (36.495%)
Indices from GENCAT matched with id:
92064 out of 232124 (39.662%)


,id_orig,title,cpv_div,index_agg,index_cat
0,(/2021,{'ampliació de lescola de freginals per a disp...,{'45': 2},[0],[0]
1,(900243/22),{'contracte de serveis per a laixecament topog...,{'71': 2},[1],[3]
2,(902472/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[2],[4]
3,(902531/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[3],[5]
4,(902766/21),{'contracte de serveis per a laixecament topog...,{'71': 2},[4],[6]


In [172]:
# Get elements that appear in gencat but not in place
cat_indices = matched_final_orig["index_cat"].explode().dropna().tolist()
use_tend_cat_orig[~use_tend_cat_orig.index.isin(cat_indices)]

,id_orig,index,title,cpv_div
1,(2) 49/2021/cmsubm2,[1],compra del vestuari i dels equips de protecció...,18
2,(2)120/2021/cmsubm1,[2],subministrament de productes dhorticultura i v...,03
8,(apc/fo/contractacio/2017/32,[8],serveis de formació de 170 hores dels mòduls m...,NaN
9,(bis) 114/2021/cmserv1,[9],contracte admnistratiu de serveis de serveis r...,79
10,(bis) 12/2021/cmsubm2,[10],productes de lagricultura ramaderia pesca silv...,03
...,...,...,...,...
232106,xx-xx,[232106],acord marc del subministrament dantiseptics i ...,NaN
232107,xxx,[232107],xxx_test,45
232121,zls23-0084,[232121],2023 1r trimestre contractes menors associació...,79
232122,zls23-0127,[232122],2023 1r trimestre contractes menors associació...,79


In [169]:
tender_filt[
    tender_filt["id_orig"]
    .str.lower()
    .apply(
        lambda x: bool(regex.search(r".*zls23.*0084.*", x)) if not pd.isna(x) else False
    )
]

,identifier,title,summary,id,cpv,location,id_orig,id_norm,cpv_div


In [ ]:
# # Unique indices (by id)
# _tot_id = len(matched_id) / len(valid_tend_cat)
# # Total elements (taking into account some values appear multiple times)
# _tot_id_sum = matched_id["title"].apply(lambda x: sum(x.values())).sum() / len(
#     valid_tend_cat
# )

# # Unique indices (by cpv and title)
# _tot_ti = len(matched_aux) / len(valid_tend_cat)
# # Total elements (taking into account some values appear multiple times)
# _tot_ti_sum = matched_aux["title"].apply(lambda x: sum(x.values())).sum() / len(
#     valid_tend_cat
# )

# # Unique aggregated
# _tot = len(matched_final) / len(valid_tend_cat)
# # Total elements (taking into account some values appear multiple times)
# _tot_sum = matched_final["title"].apply(lambda x: sum(x.values())).sum() / len(
#     valid_tend_cat
# )

# print(f"{'Unique indices (by id): ':<40}{_tot_id:.5f}")
# print(f"{'Total elements (by id): ':<40}{_tot_id_sum:.5f}")
# print()
# print(f"{'Unique indices (by cpv and title): ':<40}{_tot_ti:.5f}")
# print(f"{'Total elements (by cpv and title): ':<40}{_tot_ti_sum:.5f}")
# print()
# print(f"{'Unique indices (aggregated): ':<40}{_tot:.5f}")
# print(f"{'Total elements (aggregated): ':<40}{_tot_sum:.5f}")